In [1]:
# Import packages
import numpy as np
import pandas as pd

In [2]:
### Import data

al = pd.read_csv('airlines.csv') # airlines 
ap = pd.read_csv('airports.csv') # airports
fl = pd.read_csv('flights.csv') # flights
pl = pd.read_csv('planes.csv') # planes

In [3]:
al.head()

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.


In [4]:
ap.head()

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York


In [5]:
fl.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [6]:
pl.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan


## 1. Which city is the most popular destination from New York?

In [7]:
# Count the number of flights to each city
print(fl['origin'].value_counts())

# Check if all the origins are New York airports
ap[(ap['faa'] == 'EWR') | (ap['faa'] == 'JFK') | (ap['faa'] == 'LGA')]

origin
EWR    120835
JFK    111279
LGA    104662
Name: count, dtype: int64


,faa,name,lat,lon,alt,tz,dst,tzone
460,EWR,Newark Liberty Intl,40.692500,-74.168667,18,-5,A,America/New_York
691,JFK,John F Kennedy Intl,40.639751,-73.778925,13,-5,A,America/New_York
786,LGA,La Guardia,40.777245,-73.872608,22,-5,A,America/New_York


In [8]:
# Top 5 popular destination from New York
popular_dest = fl[['dest']].value_counts().sort_values(ascending = False)
popular_dest_name = pd.merge(left=popular_dest.head(), 
                             right=ap, 
                             how='inner',
                             left_on='dest',
                             right_on='faa')
popular_dest_name

,count,faa,name,lat,lon,alt,tz,dst,tzone
0,17283,ORD,Chicago Ohare Intl,41.978603,-87.904842,668,-6,A,America/Chicago
1,17215,ATL,Hartsfield Jackson Atlanta Intl,33.636719,-84.428067,1026,-5,A,America/New_York
2,16174,LAX,Los Angeles Intl,33.942536,-118.408075,126,-8,A,America/Los_Angeles
3,15508,BOS,General Edward Lawrence Logan Intl,42.364347,-71.005181,19,-5,A,America/New_York
4,14082,MCO,Orlando Intl,28.429394,-81.308994,96,-5,A,America/New_York


## 2. Which month is the busiest one over the year?

In [9]:
# Sort the number of flights by month
fl['month'].value_counts().sort_values(ascending=False)

month
7     29425
8     29327
10    28889
3     28834
5     28796
4     28330
6     28243
12    28135
9     27574
11    27268
1     27004
2     24951
Name: count, dtype: int64

## 3. Which airline is the most punctual?

In [10]:
# Create a dataframe with carrier, departure delay and arrival delay
punctual_carrier = fl[['carrier','dep_delay','arr_delay']].fillna(value = 0)

# Calculate the total delay time for each carrier
punctual_carrier['not_punctual_time'] = punctual_carrier['dep_delay'] + punctual_carrier['arr_delay']

# Average delay time for each carrier
punctual_carrier = punctual_carrier.groupby(by = 'carrier').agg(mean_delay = ('not_punctual_time', 'mean')).sort_values('mean_delay', ascending=True).reset_index()

# Merge the airline name
pd.merge(punctual_carrier, al, how = 'inner',on = 'carrier')[['carrier','name','mean_delay']]

,carrier,name,mean_delay
0,AS,Alaska Airlines Inc.,-4.072829
1,HA,Hawaiian Airlines Inc.,-2.014620
2,US,US Airways Inc.,5.716790
3,AA,American Airlines Inc.,8.774756
4,DL,Delta Air Lines Inc.,10.826190
5,VX,Virgin America,14.540876
6,UA,United Air Lines Inc.,15.468968
7,MQ,Envoy Air,20.278365
8,OO,SkyWest Airlines Inc.,22.218750
9,B6,JetBlue Airways,22.267978


## 4. What destination is having the longest flight duration?

In [11]:
# Calculate the average airtime for each pair of origin and destination
fl_airtime = fl.groupby(['origin','dest']).agg(average_airtime = ('air_time','mean')).reset_index().sort_values(by='average_airtime', ascending=False)

# Merge the airport name
pd.merge(left=fl_airtime, right=ap[['faa','name']], left_on='dest', right_on='faa', how='inner')

,origin,dest,average_airtime,faa,name
0,JFK,HNL,623.087719,HNL,Honolulu Intl
1,EWR,HNL,612.075209,HNL,Honolulu Intl
2,EWR,ANC,413.125000,ANC,Ted Stevens Anchorage Intl
3,JFK,SFO,347.403626,SFO,San Francisco Intl
4,JFK,SJC,346.606707,SJC,Norman Y Mineta San Jose Intl
...,...,...,...,...,...
212,EWR,ALB,31.787081,ALB,Albany Intl
213,JFK,PHL,30.836872,PHL,Philadelphia Intl
214,EWR,PHL,28.666667,PHL,Philadelphia Intl
215,EWR,BDL,25.466019,BDL,Bradley Intl


## 5. Which airline is the worst in terms of delay time?

In [12]:
# Sort aveage delay time by descending
pd.merge(left=punctual_carrier, 
         right=al, 
         how='inner',
         on='carrier')[
             ['carrier','name','mean_delay']
             ].sort_values(by = 'mean_delay', ascending = False)

,carrier,name,mean_delay
15,F9,Frontier Airlines Inc.,41.919708
14,FL,AirTran Airways Corporation,37.898160
13,EV,ExpressJet Airlines Inc.,33.820409
12,YV,Mesa Airlines Inc.,31.307820
11,WN,Southwest Airlines Co.,26.902240
10,9E,Endeavor Air Inc.,22.693391
9,B6,JetBlue Airways,22.267978
8,OO,SkyWest Airlines Inc.,22.218750
7,MQ,Envoy Air,20.278365
6,UA,United Air Lines Inc.,15.468968


## 6. Which airline has the highest capacity of seats?

In [13]:
# Get carrer and tail number
carrier = fl[['carrier','tailnum']].drop_duplicates()

# Merge the carrier to the airline name
carrier_plane = pd.merge(left=carrier, 
                         right=pl[['tailnum','seats']], 
                         how='left', 
                         on='tailnum').fillna(0).groupby(by = 'carrier').agg(
                             total_seats=('seats','sum')
                             ).sort_values(
                                 by='total_seats', 
                                 ascending=False)

# Merge the carrier to the airline name
carrier_plane = pd.merge(left=carrier_plane, right=al, on='carrier', how='inner')
carrier_plane

,carrier,total_seats,name
0,UA,116252.0,United Air Lines Inc.
1,DL,115715.0,Delta Air Lines Inc.
2,WN,82700.0,Southwest Airlines Co.
3,US,57821.0,US Airways Inc.
4,AA,29309.0,American Airlines Inc.
5,B6,27148.0,JetBlue Airways
6,EV,19525.0,ExpressJet Airlines Inc.
7,9E,13685.0,Endeavor Air Inc.
8,AS,13465.0,Alaska Airlines Inc.
9,FL,13451.0,AirTran Airways Corporation


## 7. Which airplane model from which manufacturer is the highest in use?

In [14]:
# Count of flight for each plane
plane_use = fl.groupby('tailnum').agg(count_use=('tailnum', 'count')).reset_index()

# Merge the plane model and manufacturer
pd.merge(plane_use, pl[['tailnum', 'model', 'manufacturer']]).groupby(['model', 'manufacturer']).agg(total_flight = ('count_use', 'sum')).sort_values('total_flight', ascending = False).reset_index()

,model,manufacturer,total_flight
0,A320-232,AIRBUS,31278
1,EMB-145LR,EMBRAER,28027
2,ERJ 190-100 IGW,EMBRAER,23716
3,A320-232,AIRBUS INDUSTRIE,14553
4,EMB-145XR,EMBRAER,14051
...,...,...,...
142,767-201,BOEING,2
143,737-3A4,BOEING,1
144,747-451,BOEING,1
145,A330-223,AIRBUS INDUSTRIE,1
